# DRL usage example

We are going to rely on the script available in the repository root called `DRL_battery.py`. This script applies all the possibilities that Sinergym has to work with deep reinforcement learning algorithms and set parameters to everything so that we can define the training options from the execution of the script easily by a JSON file.

.. note:: For more information about how run `DRL_battery.py`, please, see [DRL documentation](https://ugr-sail.github.io/sinergym/compilation/main/pages/deep-reinforcement-learning.html#how-to-use)

In [1]:
import sys
from datetime import datetime

import gymnasium as gym
import numpy as np
import wandb
from stable_baselines3 import *
from stable_baselines3.common.callbacks import CallbackList
from stable_baselines3.common.logger import HumanOutputFormat, Logger
from stable_baselines3.common.monitor import Monitor

import sinergym
import sinergym.utils.gcloud as gcloud
from sinergym.utils.callbacks import *
from sinergym.utils.constants import *
from sinergym.utils.logger import CSVLogger, WandBOutputFormat
from sinergym.utils.rewards import *
from sinergym.utils.wrappers import *

First let's define some variables for the execution.

In [2]:
# Environment ID
environment = "Eplus-demo-v1"
# Training episodes
episodes = 4
#Name of the experiment
experiment_date = datetime.today().strftime('%Y-%m-%d_%H:%M')
experiment_name = 'SB3_DQN-' + environment + \
    '-episodes-' + str(episodes)
experiment_name += '_' + experiment_date

We can combine this experiment executions with [Weights&Biases](https://ugr-sail.github.io/sinergym/compilation/main/pages/deep-reinforcement-learning.html#weights-and-biases-structure) in order to host all information extracted. With *wandb*, it’s possible to track and visualize all DRL training in real time, register hyperparameters and details of each experiment, save artifacts such as models and *sinergym* output, and compare between different executions.

In [3]:
# Create wandb.config object in order to log all experiment params
experiment_params = {
    'sinergym-version': sinergym.__version__,
    'python-version': sys.version
}
experiment_params.update({'environment':environment,
                          'episodes':episodes,
                          'algorithm':'SB3_DQN'})

# Get wandb init params (you have to specify your own project and entity)
wandb_params = {"project": 'sinergym',
                "entity": 'alex_ugr'}
# Init wandb entry
run = wandb.init(
    name=experiment_name + '_' + wandb.util.generate_id(),
    config=experiment_params,
    ** wandb_params
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alex_ugr. Use `wandb login --relogin` to force relogin


Now we are ready to create the Gym Environment. Here we use the environment name defined, remember that you can [change default environment configuration](https://ugr-sail.github.io/sinergym/compilation/main/pages/notebooks/change_environment.html#Changing-an-environment-registered-in-Sinergym). We will create a eval_env too in order to interact the evaluation episodes there.

In [4]:
env = gym.make(environment)
eval_env = gym.make(environment)

[2023-04-05 11:38:26,010] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-04-05 11:38:26,012] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-04-05 11:38:26,016] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-04-05 11:38:26,019] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
[2023-04-05 11:38:26,020] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Setting up action definition in building model if exists...
[2023-04-05 11:38:27,346] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-04-05 11:38:27,346] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-04-05 11:38:27,348] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating

We can also add a Wrapper to the environment, we are going to use a Logger (extension of `gym.Wrapper`) this is used to monitor and log the interactions with the environment and save the data into a CSV. Files generated will be stored as artifact in *wandb* too.

In [5]:
env = LoggerWrapper(env)

At this point, we have the environment set up and ready to be used. We are going to create our learning model (Stable Baselines 3 DQN), but we can use any other algorithm.

In [6]:
model = DQN('MlpPolicy', env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


Now we need to calculate the number of timesteps of each episode for the evaluation. Evaluation will execute the current model during a number of episodes determined to decide if it is the best current version of the model. Output generated will be stored in *wandb* server too.

In [8]:
n_timesteps_episode = env.simulator._eplus_one_epi_len / \
                      env.simulator._eplus_run_stepsize


We are going to use the LoggerEval callback to print and save the best model evaluated during training.

In [9]:
callbacks = []

# Set up Evaluation and saving best model
eval_callback = LoggerEvalCallback(
    eval_env,
    best_model_save_path=eval_env.simulator._env_working_dir_parent +
    '/best_model/',
    log_path=eval_env.simulator._env_working_dir_parent +
    '/best_model/',
    eval_freq=n_timesteps_episode * 2,
    deterministic=True,
    render=False,
    n_eval_episodes=1)
callbacks.append(eval_callback)

callback = CallbackList(callbacks)

In order to track all the training process in *wandb*, it is necessary to create a callback with a compatible wandb output format (which call *wandb* log in the learning algorithm process).

In [12]:
# wandb logger and setting in SB3
logger = Logger(
    folder=None,
    output_formats=[
        HumanOutputFormat(
            sys.stdout,
            max_length=120),
        WandBOutputFormat()])
model.set_logger(logger)
# Append callback
log_callback = LoggerCallback()
callbacks.append(log_callback)


callback = CallbackList(callbacks)

This is the number of total time steps for the training.

In [10]:
timesteps = episodes * n_timesteps_episode

Now is time to train the model with the callbacks defined earlier. This may take a few minutes, depending on your computer.

In [13]:
model.learn(
    total_timesteps=timesteps,
    callback=callback,
    log_interval=1)

[2023-04-05 11:40:26,600] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-04-05 11:40:26,600] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-04-05 11:40:26,800] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-demo-v1-res1/Eplus-env-sub_run1
[2023-04-05 11:40:26,800] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-demo-v1-res1/Eplus-env-sub_run1


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[2023-04-05 11:41:05,587] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-04-05 11:41:05,587] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-04-05 11:41:05,590] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-04-05 11:41:05,590] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-04-05 11:41:05,743] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-demo-v1-res1/Eplus-env-sub_run2
[2023-04-05 11:41:05,743] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-demo-v1-res1/Eplus-env-sub_run2
-------------------------------------------------------------------------------------
| action/                                                             |             |
|    Cooling_Setpoint_RL                    

Now we save the current model (model version when training has finished).

In [14]:
model.save(env.simulator._env_working_dir_parent + '/' + experiment_name)

And as always, remember to close the environment.

In [15]:
env.close()

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[2023-04-05 11:51:41,481] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-04-05 11:51:41,481] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 


We have all the experiments results in our local computer, but we can see the execution in *wandb* too:


- Hyperparameter and summary registration:

![wandb_example1](https://github.com/ugr-sail/sinergym/blob/main/docs/source/_static/wandb_example1.png?raw=true)



- Artifacts registered (if evaluation is enabled, best model is registered too):
  
![wandb_example2](https://github.com/ugr-sail/sinergym/blob/main/docs/source/_static/wandb_example1.png?raw=true)



- Metrics visualization in real time:
  
![wandb_example3](https://github.com/ugr-sail/sinergym/blob/main/docs/source/_static/wandb_example1.png?raw=true)